# Import

In [1]:
%load_ext autoreload
%autoreload 2

import importlib
import numpy as np
import math
import itertools 
import pandas as pd
import numpy as np
import io
import msoffcrypto
from sklearn.metrics import r2_score
import openpyxl
import plotly.express as px
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import plotly
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import streamlit as st
import json
from math import factorial as fact
from IPython.display import Markdown, display
import networkx as nx
import requests
import toml
import sys, os
import collections
from types import SimpleNamespace
import copy

%matplotlib inline 
%config Completer.use_jedi = False

from utils import *

# Load

In [ ]:
with open('VALEO_full.tmj', 'r') as f:
      data = json.load(f)

In [ ]:
np.random.choice([1,2,3],2, replace=False)

In [ ]:
np.random.choice()

In [ ]:
dfline  = pd.DataFrame(data['layers'][2]['objects']).drop(columns=['rotation','width','name','height','visible']).rename(columns = {'class' : 'Class'})
idx= 0
row = dfline.iloc[idx]
properties = row.properties
properties = properties[:0] + properties[-2:]
dfline.loc[idx, ['end','start']] = [d['value'] for d in properties] 

In [ ]:
properties[(0,2)]

In [ ]:
properties.pop(1)

In [ ]:
[d['value'] for d in properties ] 

# RUN 

In [27]:
%%time
file = 'VALEO_full.tmj'
algo = load_data_brut(file)
df1 = indiv_init(algo, 50)

begin
Wall time: 95.3 ms


In [3]:
indiv_create(algo)

{'D': {'C': [0, 1, 2, 3], 'E': [0, 1, 2], 'P': [0, 1]},
 'Clist': [0, 1, 2, 3],
 'CtoE': [0, 2, 2, 1],
 'Econnect': {0: [0], 1: [3], 2: [1, 2]},
 'Elist': [0, 1, 2],
 'Ecount': 3,
 'EtoP': [0, 1, 1],
 'Pconnect': {0: [0], 1: [1, 2]},
 'Plist': [0, 1],
 'Pcount': 2,
 'List_EtoC': [['E0-C0'], ['E1-C3'], ['E2-C1', 'E2-C2']],
 'List_PtoE': [['P0-E0'], ['P1-E1', 'P1-E2']],
 'dist': 500,
 'Name': ['E0-C0', 'E1-C3', 'E2-C1', 'E2-C2', 'P0-E0', 'P1-E1', 'P1-E2']}

In [28]:
df1 = algo.df
dfs = algo.dfslot
dfline = algo.dfline

In [ ]:
for 

In [36]:
dfline[['dist','ID']].to_clipboard()

In [35]:
df1.drop(columns= ['D','Name', 'Name_txt']).to_clipboard()

In [30]:
dfs
dfline
df1.drop(columns= ['D','Name', 'Name_txt'])

,Class,id,Name,x,y,ID,Color
0,C,41,3,3,9,C3,10
1,C,38,2,6,2,C2,10
2,C,40,1,0,2,C1,10
5,E,42,1,4,2,E1,20
6,E,43,0,2,7,E0,20
7,P,44,1,2,2,P1,30


,Class,polyline,end,long,start,dist,ID
1,C-E,"[[12, 40], [20, 40], [20, 104], [36, 104], [36...",0,3.0,1,100,E0-C1
2,C-E,"[[100, 44], [88, 44], [88, 120], [44, 120]]",0,3.0,2,132,E0-C2
3,C-E,"[[52, 148], [52, 124], [44, 124]]",0,2.0,3,32,E0-C3
5,C-E,"[[12, 36], [24, 36], [24, 28], [68, 28], [68, ...",1,2.0,1,72,E1-C1
6,C-E,"[[100, 36], [76, 36]]",1,1.0,2,24,E1-C2
7,C-E,"[[56, 148], [56, 56], [72, 56], [72, 44]]",1,2.0,3,120,E1-C3
15,E-P,"[[40, 116], [40, 44]]",1,2.0,0,72,P1-E0
16,E-P,"[[68, 40], [44, 40]]",1,1.0,1,24,P1-E1


,Clist,CtoE,Econnect,Elist,Ecount,EtoP,Pconnect,Plist,Pcount,List_EtoC,List_PtoE,dist
0,"[1, 2, 3]","[1, 1, 0]","{0: [3], 1: [1, 2]}","[0, 1]",2,"[1, 1]","{1: [0, 1]}",[1],1,"[[E0-C3], [E1-C1, E1-C2]]","[[P1-E0, P1-E1]]",448
1,"[1, 2, 3]","[1, 1, 1]","{1: [1, 2, 3]}",[1],1,[1],{1: [1]},[1],1,"[[E1-C1, E1-C2, E1-C3]]",[[P1-E1]],480
2,"[1, 2, 3]","[0, 1, 0]","{0: [1, 3], 1: [2]}","[0, 1]",2,"[1, 1]","{1: [0, 1]}",[1],1,"[[E0-C1, E0-C3], [E1-C2]]","[[P1-E0, P1-E1]]",504
3,"[1, 2, 3]","[1, 0, 0]","{0: [2, 3], 1: [1]}","[0, 1]",2,"[1, 1]","{1: [0, 1]}",[1],1,"[[E0-C2, E0-C3], [E1-C1]]","[[P1-E0, P1-E1]]",664
4,"[1, 2, 3]","[0, 0, 0]","{0: [1, 2, 3]}",[0],1,[1],{1: [0]},[1],1,"[[E0-C1, E0-C2, E0-C3]]",[[P1-E0]],672
5,"[1, 2, 3]","[0, 1, 1]","{0: [1], 1: [2, 3]}","[0, 1]",2,"[1, 1]","{1: [0, 1]}",[1],1,"[[E0-C1], [E1-C2, E1-C3]]","[[P1-E0, P1-E1]]",680
6,"[1, 2, 3]","[1, 0, 1]","{0: [2], 1: [1, 3]}","[0, 1]",2,"[1, 1]","{1: [0, 1]}",[1],1,"[[E0-C2], [E1-C1, E1-C3]]","[[P1-E0, P1-E1]]",840
7,"[1, 2, 3]","[0, 0, 1]","{0: [1, 2], 1: [3]}","[0, 1]",2,"[1, 1]","{1: [0, 1]}",[1],1,"[[E0-C1, E0-C2], [E1-C3]]","[[P1-E0, P1-E1]]",896


In [20]:
DropList = ['C0','E2']
'|'.join(DropList)

'C0|E2'

In [23]:
dfline[~dfline.ID.str.contains('|'.join(DropList))]

dfs[~dfs.ID.isin(DropList)]

,Class,polyline,end,long,start,dist,ID
1,C-E,"[[12, 40], [20, 40], [20, 104], [36, 104], [36...",0,3.0,1,100,E0-C1
2,C-E,"[[100, 44], [88, 44], [88, 120], [44, 120]]",0,3.0,2,132,E0-C2
3,C-E,"[[52, 148], [52, 124], [44, 124]]",0,2.0,3,32,E0-C3
5,C-E,"[[12, 36], [24, 36], [24, 28], [68, 28], [68, ...",1,2.0,1,72,E1-C1
6,C-E,"[[100, 36], [76, 36]]",1,1.0,2,24,E1-C2
7,C-E,"[[56, 148], [56, 56], [72, 56], [72, 44]]",1,2.0,3,120,E1-C3
12,E-P,"[[36, 120], [28, 120]]",0,1.0,0,8,P0-E0
13,E-P,"[[68, 44], [68, 52], [28, 52], [28, 116]]",0,3.0,1,112,P0-E1
15,E-P,"[[40, 116], [40, 44]]",1,2.0,0,72,P1-E0
16,E-P,"[[68, 40], [44, 40]]",1,1.0,1,24,P1-E1


,Class,id,Name,x,y,ID,Color
0,C,41,3,3,9,C3,10
1,C,38,2,6,2,C2,10
2,C,40,1,0,2,C1,10
5,E,42,1,4,2,E1,20
6,E,43,0,2,7,E0,20
7,P,44,1,2,2,P1,30
8,P,45,0,1,7,P0,30


In [5]:
pd.DataFrame(algo.Comb)

ValueError: All arrays must be of the same length

In [51]:
indiv_init(algo, 10)

,D,CtoE,Econnect,Elist,Ecount,EtoP,Pconnect,Plist,Pcount,List_EtoC,List_PtoC,dist,Name
1,"{'C': [1, 2, 3], 'E': [0, 1], 'P': [0, 1]}","[1, 0, 0]","{0: [2, 3], 1: [1]}","[0, 1]",2,"[0, 1]","{0: [0], 1: [1]}","[0, 1]",2,"[[E0-C2, E0-C3], [E1-C1]]","[[P0-E0], [P1-E1]]",280,"E0-C2,E0-C3,E1-C1,P0-E0,P1-E1"
4,"{'C': [1, 2, 3], 'E': [0, 1], 'P': [0, 1]}","[0, 1, 0]","{0: [1, 3], 1: [2]}","[0, 1]",2,"[0, 0]","{0: [0, 1]}",[0],1,"[[E0-C1, E0-C3], [E1-C2]]","[[P0-E0, P0-E1]]",284,"E0-C1,E0-C3,E1-C2,P0-E0,P0-E1"
0,"{'C': [1, 2, 3], 'E': [0, 1], 'P': [0, 1]}","[0, 1, 1]","{0: [1], 1: [2, 3]}","[0, 1]",2,"[0, 0]","{0: [0, 1]}",[0],1,"[[E0-C1], [E1-C2, E1-C3]]","[[P0-E0, P0-E1]]",300,"E0-C1,E1-C2,E1-C3,P0-E0,P0-E1"
5,"{'C': [1, 2, 3], 'E': [0, 1], 'P': [0, 1]}","[0, 0, 0]","{0: [1, 2, 3]}",[0],1,[1],{1: [0]},[1],1,"[[E0-C1, E0-C2, E0-C3]]",[[P1-E0]],348,"E0-C1,E0-C2,E0-C3,P1-E0"
3,"{'C': [1, 2, 3], 'E': [0, 1], 'P': [0, 1]}","[0, 1, 1]","{0: [1], 1: [2, 3]}","[0, 1]",2,"[1, 0]","{0: [1], 1: [0]}","[0, 1]",2,"[[E0-C1], [E1-C2, E1-C3]]","[[P0-E1], [P1-E0]]",364,"E0-C1,E1-C2,E1-C3,P0-E1,P1-E0"
9,"{'C': [1, 2, 3], 'E': [0, 1], 'P': [0, 1]}","[1, 0, 0]","{0: [2, 3], 1: [1]}","[0, 1]",2,"[0, 0]","{0: [0, 1]}",[0],1,"[[E0-C2, E0-C3], [E1-C1]]","[[P0-E0, P0-E1]]",368,"E0-C2,E0-C3,E1-C1,P0-E0,P0-E1"
7,"{'C': [1, 2, 3], 'E': [0, 1], 'P': [0, 1]}","[0, 0, 1]","{0: [1, 2], 1: [3]}","[0, 1]",2,"[1, 1]","{1: [0, 1]}",[1],1,"[[E0-C1, E0-C2], [E1-C3]]","[[P1-E0, P1-E1]]",388,"E0-C1,E0-C2,E1-C3,P1-E0,P1-E1"
2,"{'C': [1, 2, 3], 'E': [0, 1], 'P': [0, 1]}","[0, 0, 1]","{0: [1, 2], 1: [3]}","[0, 1]",2,"[0, 0]","{0: [0, 1]}",[0],1,"[[E0-C1, E0-C2], [E1-C3]]","[[P0-E0, P0-E1]]",412,"E0-C1,E0-C2,E1-C3,P0-E0,P0-E1"
6,"{'C': [1, 2, 3], 'E': [0, 1], 'P': [0, 1]}","[1, 0, 1]","{0: [2], 1: [1, 3]}","[0, 1]",2,"[1, 0]","{0: [1], 1: [0]}","[0, 1]",2,"[[E0-C2], [E1-C1, E1-C3]]","[[P0-E1], [P1-E0]]",448,"E0-C2,E1-C1,E1-C3,P0-E1,P1-E0"


In [ ]:
SimpleNamespace(**l)

In [ ]:
for l in df.values.tolist():
    col = ['D', 'CtoE','Econnect','Elist','Ecount', 'EtoP','Pconnect','Plist','Pcount', 'List_EtoC','List_PtoC', 'dist', 'Name']

In [ ]:
indivs = []

In [ ]:
dfline.sort_values('ID').reset_index(drop = True)
dfslot

# Test combinatoire sequencage 

In [ ]:
%%time
data, dfs, dfline , df1 = load_data_brut(500)
df1.shape

In [ ]:
df1

In [ ]:
index = 0
row = df1.loc[index]

D = dfs.Class.value_counts().to_dict()
Clist = list(range(D['C']))
CtoE = row.CtoE
Econnect = row.Econnect
ID_CtoE = ['C{}-E{}'.format(i, CtoE[i]) for i in Clist]
#     ID_CtoE

d = collections.defaultdict(list)
for i in Clist:
    c = CtoE[i]
    d[c].append(i)
d.items()

In [ ]:
N = 4
a = np.zeros(N, dtype= int)
a = np.array([0,0,1,0])
a
a==0
g1 = np.max(a)+1
g1
idx = np.random.randint(N)
idx
idx = 2
np.random.randint(g1)

In [ ]:
x1 = np.array([True,False])
x2 = np.array([True,True])
True & x1 & x2

In [ ]:
x1 = np.array([0, 0, 0, 1])
(x1 == 1).sum()!=1

In [ ]:
N = 3

a = np.array(list(itertools.product(range(N), repeat= N+1)))
a.shape
m = (a==0).sum(1)
# m
m2 = True
for i in range(1,N):
    print(i)
    m2 =  
m2
np.vstack([a.T,m2]).T

In [ ]:
N = 3

a0 = np.array(list(itertools.product(range(N), repeat= N+1)))


# m = (a==0).sum(1)
# m
m2 = False
M = []
for i in range(1,N):
#     print(i)
    m2 = m2 | (a0==i).sum(1) == 1
    M.append(m2)
# m2
M.append(~m2)
# m = (m == 2|3)
# m = (m == N)
a = a0[~m2]
a0.shape , a.shape
a

In [ ]:
Liste = [a0.T] + M
np.vstack(Liste).T

In [ ]:
Ecomb = a[1]
Ecomb
collections.Counter(Ecomb)

In [ ]:
dict(collections.Counter(Ecomb))

In [ ]:
d = collections.defaultdict(list)
for n,i in enumerate(Ecomb):
    d[i].append(n)
dict(d.items())

In [ ]:
np.where((arr==0)|(arr==1), arr^1, arr)

In [ ]:
L = []
for idx , row in df1.iterrows():
    index = 0
    row = df1.loc[index]

    D = dfs.Class.value_counts().to_dict()
    Clist = list(range(D['C']))
    CtoE = row.CtoE
    Econnect = row.Econnect
    ID_CtoE = ['C{}-E{}'.format(i, CtoE[i]) for i in Clist]
#     ID_CtoE
    
    d = collections.defaultdict(list)
    for i in Clist:
        c = CtoE[i]
        d[c].append(i)
#     d.items()

    for k,v in d.items(): 
#         print(k,v)
        for i in range(len(v)):
            l2 = list(itertools.combinations(v, i+1))
            L.append(row.tolist() + [l2])
    break
            
#             print(l2)

In [ ]:
df1.columns.tolist() + ['Ecomb']

In [ ]:
pd.DataFrame(L, columns= df1.columns.tolist() + ['Ecomb'])

In [ ]:
index = 0
row = df1.loc[index]
row

D = dfs.Class.value_counts().to_dict()
Clist = list(range(D['C']))
CtoE = row.CtoE
Econnect = row.Econnect
ID_CtoE = ['C{}-E{}'.format(i, CtoE[i]) for i in Clist]
ID_CtoE

In [ ]:
d = collections.defaultdict(list)
for i in Clist:
    c = CtoE[i]
    d[c].append(i)
d.items()

for k,v in d.items(): 
    print(k,v)
    for i in range(len(v)):
        l2 = list(itertools.combinations(v, i+1))
        print(l2)

In [ ]:
list(itertools.product(([0,1]), repeat=2))

In [ ]:
d = {}
for i in Clist:
    c = CtoE[i]
    d[c].append(i)
d.items()

In [ ]:
l = list(zip(row.CtoE, row.Clist))
d = collections.defaultdict(list)
for k, v in l:
    d[k].append(v)
d.items()

In [ ]:
D = dfs.Class.value_counts().to_dict()
Clist = list(range(D['C']))
CtoE = row.CtoE
Econnect = row.Econnect
ID_CtoE = ['C{}-E{}'.format(i, CtoE[i]) for i in Clist]
ID_CtoE

In [ ]:
index = 0
row = df1.loc[index]

# print(row)
# st._legacy_dataframe(row.to_frame().T)
ElemsList = ['Clist','Elist','Plist']
Elems = ['C','E','P']
IDSelects = []
ID_CtoE = row.ID_CtoE
ID_EtoP = row.ID_EtoP
for n in range(3):
    IDSelects+= ['{}{}'.format(Elems[n],i) for i in row[ElemsList[n]]]

dflineSelect = dfline[dfline.ID.isin(ID_CtoE + ID_EtoP)].copy()
dfsSelect = dfs[dfs.ID.isin(IDSelects)].copy()

In [ ]:
fig = plot_(data,dflineSelect, dfsSelect)

In [ ]:
fig = plot_(data,dfline, dfs)

# Tiled import

## func

In [ ]:
data, dfs, dfline , df1 = load_data_brut(10)

In [ ]:

df1

In [ ]:
dfx.

In [ ]:
row1

In [ ]:
df1.sample(10)

In [ ]:
ListSelectbox = df1.index
index = ListSelectbox[0]
row = df1.loc[index]
row

ElemsList = ['Clist','Elist','Plist']
Elems = ['C','E','P']
IDSelects = []
ID_CtoE = row.ID_CtoE
ID_EtoP = row.ID_EtoP
for n in range(3):
    IDSelects+= ['{}{}'.format(Elems[n],i) for i in row[ElemsList[n]]]

dflineSelect = dfline[dfline.ID.isin(ID_CtoE + ID_EtoP)].copy()
dfsSelect = dfs[dfs.ID.isin(IDSelects)].copy()

In [ ]:
row.to_clipboard()

In [ ]:
dflineSelect

In [ ]:
dflineSelect.drop(columns= ['long', 'polyline']).to_clipboard()

In [ ]:
dfsSelect.to_clipboard()

In [ ]:
fig = plot_(data,dflineSelect, dfsSelect)

## Algo gen 

In [ ]:
def Reprodution(dfx,dfs):
    c1, c2 = copy.deepcopy(dfx.CtoE.values)
    
    if (c1 != c2).any():        
        m = c1 != c2
        index = np.where(m)[0]
        
        if len(index) > 1:    
            idx = np.random.choice(index)
        elif len(index) ==1:
            idx = index            
#         print(i1,i2,c1, c2,m, idx)    
        c1[idx] , c2[idx] = c2[idx] , c1[idx]
        NewCtoE = c1 , c2
#         print(NewCtoE)
        L = []

        for i in range(2):
            row = dfx.iloc[i]
#             print(row.to_list())
            l = indiv_verif(row,NewCtoE[i],dfs) 
#             print(l) 
            L.append(l)
        return L

In [ ]:
def indiv_verif(row,NewCtoE,dfs): 
    D = dfs.Class.value_counts().to_dict()
    Clist = list(range(D['C']))
    
    CtoE = NewCtoE
    Econnect = dict(collections.Counter(CtoE))
    Elist = sorted(Econnect)
    Ecount = len(Elist)   
    
    if Ecount > row.Ecount:
        x = row.EtoP
        y = np.random.randint(0,D['P'],Ecount - row.Ecount)
        EtoP = np.append(x, y)
    elif Ecount < row.Ecount:
        EtoP = np.random.choice(row.EtoP,row.Ecount - Ecount)
        print(EtoP)
    else : 
        EtoP = row.EtoP

    Pconnect = dict(collections.Counter(EtoP))
    Plist = sorted(Pconnect)
    Pcount = len(Plist)    

    ID_CtoE = ['C-E{}{}'.format(i, CtoE[i]) for i in Clist]
    ID_EtoP = ['E-P{}{}'.format(Elist[i]  , EtoP[i]) for i in range(Ecount)]

    dist = dfline.loc[dfline.ID.isin(ID_CtoE + ID_EtoP), 'dist'].sum()  
    l = [Clist, CtoE,Econnect,Elist,Ecount, EtoP,Pconnect,Plist,Pcount, ID_CtoE,ID_EtoP,dist]
    return l 


In [ ]:
data, dfs, dfline , df1 = load_data_brut(7)
df1['Epoch'] = df1.index
Epoch = df1.index.max()+1
df1

In [ ]:
L = []
List = df1[:7].index.values
np.random.shuffle(List)
print(List)
for n in range(3):
    print(n)
    i1 = List[n*2]
    i2 = List[n*2 + 1]
    dfx = df1.loc[[i1,i2]].copy()
    L2 = Reprodution(dfx, dfs)
    if L2:
        for l in L2:
            l.append(Epoch)
            l.append(dfx.Epoch.tolist())
            Epoch+= 1
            L.append(l)
#     print(n,Epoch, L2)
#             print(l, n)

In [ ]:
col = ['Clist', 'CtoE','Econnect','Elist','Ecount', 'EtoP','Pconnect','Plist','Pcount', 'ID_CtoE','ID_EtoP','dist','Epoch','Parents']
dfx = pd.DataFrame(L, columns= col)
# dfx
df1 = df1.append(dfx)
df1['Name'] = (df1.ID_CtoE + df1.ID_EtoP).str.join(',')
df1 = df1.drop_duplicates(subset='Name').sort_values('dist').reset_index(drop = True)

In [ ]:
df1

## Data

In [ ]:
with open('VALEO_1.tmj', 'r') as f:
  data = json.load(f)

In [ ]:
dfs = pd.DataFrame(data['layers'][1]['objects']).drop(columns= ['rotation','width','height','visible','gid']).rename(columns = {'class' : 'Class', 'name' : 'Name'})
dfs.x = (dfs.x/16).astype(int)
dfs.y = (dfs.y/16).astype(int) - 1
dfs.Name = pd.to_numeric(dfs.Name)
dfs['ID'] = dfs.Class + dfs.Name.astype(str)
dfs['Color'] = dfs['Class'].map({'C':10,'E':20,'P':30})

dfs

In [ ]:
dfline  = pd.DataFrame(data['layers'][2]['objects']).drop(columns=['rotation','width','name','height','visible']).rename(columns = {'class' : 'Class'})

for idx, row in dfline.iterrows():
    properties = row.properties
    dfline.loc[idx, ['end','long','start']] = [d['value'] for d in properties]    
    polyline = row.polyline
    x = row.x
    y = row.y
    polyline = [(p['x'] + x, p['y'] + y) for p in polyline]
    p = np.array(polyline)
    dfline.at[idx , 'polyline'] = p   
    dfline.loc[idx ,'dist'] = np.abs(np.diff(p.T)).sum()
dfline.start = pd.to_numeric(dfline.start)
dfline.end = pd.to_numeric(dfline.end)
dfline.dist = dfline.dist.astype(int)
dfline = dfline.drop(columns= ['properties','x','y'])
dfline['ID'] = dfline.Class + dfline.start.astype(str) + dfline.end.astype(str)
dfline

## plot all

In [ ]:
N = data['height']
A0 = data['layers'][0]['data']
A0 = np.array(A0).reshape(10,7)
pas = 16
unique = np.unique(A0)
A0[A0 == unique[0]] = 0
A0[A0 == unique[1]] = 1
A0

In [ ]:
for idx, row in dfs.iterrows():
#     print(row.x, row.y , row.Type * int(row.name))
    A0[row.y, row.x] = row.Type + int(row.Name)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(A0)

In [ ]:
A = np.kron(A0, np.ones((16,16), dtype=int))
fig, ax = plt.subplots(figsize = (10,10))
ax.imshow(A)
for idx, row in dfline.iterrows():
    p = np.array(row.polyline)
    f = ax.plot(p[:,0],p[:,1],'k')
style = dict(size=10, color='black')  
for idx, row in dfs.iterrows():
    x = row.x*16
    y = row.y*16
    text = row.Class + str(row.Name)
    f = ax.text(row.x*16+8, row.y*16+8,text , **style,  ha='center', weight='bold') 

## indiv

## Objectif
269ms value count VS 9 ms collection & np bincount

In [ ]:
D = dfs.Class.value_counts().to_dict()
D

In [ ]:
%%time
L = []
D = dfs.Class.value_counts().to_dict()

for n in range(1000): 
    Clist = list(range(D['C']))

    CtoE = np.random.randint(0,D['E'],D['C'])
    Econnect = collections.Counter(CtoE)
    Elist = sorted(Econnect)
    Ecount = len(Elist)

    EtoP = np.random.randint(0,D['P'],Ecount)
    Pconnect = collections.Counter(EtoP)
    Plist = sorted(Pconnect)
    Pcount = len(Plist)

    ID_CtoE = ['C-E{}{}'.format(i, CtoE[i]) for i in Clist]
    ID_EtoP = ['E-P{}{}'.format(Elist[i]  , EtoP[i]) for i in range(Ecount)]

    dist = dfline.loc[dfline.ID.isin(ID_CtoE + ID_EtoP), 'dist'].sum()

    l = [Clist, CtoE,Econnect,Elist,Ecount, EtoP,Pconnect,Plist,Pcount, ID_CtoE,ID_EtoP,dist]
    L.append(l)  

In [ ]:
col = ['Clist', 'CtoE','Econnect','Elist','Ecount', 'EtoP','Pconnect','Plist','Pcount', 'ID_CtoE','ID_EtoP','dist']
df = pd.DataFrame(L, columns= col)
df.sort_values('dist').head(10)

In [ ]:
df['Name'] = (df.ID_CtoE + df.ID_EtoP).str.join(',')
df1 = df.drop_duplicates(subset='Name').sort_values('dist')

In [ ]:
df1

In [ ]:
ListSelectbox = df1[['ID_CtoE', 'ID_EtoP','dist']].values

In [ ]:
df1

In [ ]:
ListSelectbox = df1.index
index = ListSelectbox[0]
row = df1.loc[index]
row

Select = ListSelectbox[0]
Select
name = ','.join(Select[0]+Select[1])

row = df1[df1.Name == name].iloc[0]
row

row = df.iloc[3]
row

In [ ]:
ElemsList = ['Clist','Elist','Plist']
Elems = ['C','E','P']
IDSelects = []
ID_CtoE = row.ID_CtoE
ID_EtoP = row.ID_EtoP
for n in range(3):
    IDSelects+= ['{}{}'.format(Elems[n],i) for i in row[ElemsList[n]]]

dflineSelect = dfline[dfline.ID.isin(ID_CtoE + ID_EtoP)].copy()
dfsSelect = dfs[dfs.ID.isin(IDSelects)].copy()

In [ ]:
dflineSelect

In [ ]:
dfsSelect

## plot

In [ ]:
def plot_(data,dflineSelect, dfsSelect): 
    N = data['height']
    A0 = data['layers'][0]['data']
    A0 = np.array(A0).reshape(10,7)
    pas = 16
    unique = np.unique(A0)
    A0[A0 == unique[0]] = 0
    A0[A0 == unique[1]] = 1
    for idx, row in dfsSelect.iterrows():
        A0[row.y, row.x] = row.Color + int(row.Name)   
        
    A = np.kron(A0, np.ones((16,16), dtype=int))
    fig, ax = plt.subplots(figsize = (10,10))
    ax.imshow(A)
    for idx, row in dflineSelect.iterrows():
        p = row.polyline
        f = ax.plot(p[:,0],p[:,1],'k')

    style = dict(size=10, color='black') 
    for idx, row in dfsSelect.iterrows():
        x = row.x*16
        y = row.y*16
        text = row.Class + str(row.Name)
        f = ax.text(row.x*16+8, row.y*16+8,text , **style,  ha='center', weight='bold') 
    return fig

In [ ]:
fig = plot_(data,dflineSelect, dfsSelect)

## Plot save

In [ ]:
%%time
R = np.random.randint(0,D['EV'],D['C'])
dfline['Select'] = False
dfs['Connect'] = 0
dfs.loc[dfs.Class == 'C', 'Connect']= 1
for i in range(D['C']):
    j = R[i]
    mask = (dfline.Class == 'C-E') & (dfline.start == i) & (dfline.end == j)
#     print(i,j, mask.sum())
    dfline.loc[mask, 'Select'] = True 
    
    mask = (dfs.Class == 'EV') & (dfs.Name == j)
    dfs.loc[mask, 'Connect'] +=  + 1
    
for i in np.unique(R):
    j = np.random.randint(0,D['TP'])
    mask = (dfline.Class == 'P-E') & (dfline.start == j) & (dfline.end == i)
#     print(i,j, mask.sum())
    dfline.loc[mask, 'Select'] = True
    
    mask = (dfs.Class == 'TP') & (dfs.Name == j)
    dfs.loc[mask, 'Connect'] +=  + 1
    
dfline  
dfs

In [ ]:
N = data['height']
A0 = data['layers'][0]['data']
A0 = np.array(A0).reshape(10,7)
pas = 16
unique = np.unique(A0)
A0[A0 == unique[0]] = 0
A0[A0 == unique[1]] = 1
for idx, row in dfs.iterrows():
    if row.Connect > 0:
        A0[row.y, row.x] = row.Color + int(row.Name)
A0        

In [ ]:
A = np.kron(A0, np.ones((16,16), dtype=int))
fig, ax = plt.subplots(figsize = (10,10))
ax.imshow(A)
for idx, row in dfline.iterrows():
    if row.Select: 
        p = np.array(row.polyline)
        f = ax.plot(p[:,0],p[:,1],'k')

style = dict(size=10, color='black') 
for idx, row in dfs.iterrows():
    if row.Connect > 0:
        x = row.x*16
        y = row.y*16
        text = row.Class + str(row.Name)
        f = ax.text(row.x*16+8, row.y*16+8,text , **style,  ha='center', weight='bold') 

## Calcul distance

In [ ]:
for idx, row in dfline.iterrows():
    p = np.array(row.polyline)
    dist = np.abs(np.diff(p.T)).sum()
    print(dist)

In [ ]:
dfline

In [ ]:
row = dfline.iloc[4]
p = np.array(row.polyline)
row.polyline
p.T
np.diff(p.T)
np.abs(np.diff(p.T))
np.abs(np.diff(p.T)).sum()

## Save

In [ ]:
Elist = np.unique(R)
Elist
R = np.random.randint(0,D['EV'],D['C'])

In [ ]:
pd.value_counts(R).sort_index()

In [ ]:
ci = pd.DataFrame({'EV':ce}).T
ci.columns.name = 'C'
ci

In [ ]:
ep = pd.value_counts(ce).sort_index()
ep.name = 'Count'
ep= ep.to_frame()
ep

ep['TP'] = np.random.randint(0,D['TP'],len(ep))
ep

In [ ]:
ci
ep.T

In [ ]:
indivs = {}
for i in range(10):
    r = np.random.randint()
    
    break
    
    
    

In [ ]:
np.unique(C,
          return_index=True,
          return_inverse=True,
          return_counts=True,axis = 1)

In [ ]:

np.unique(E[0],
          return_index=True,
          return_inverse=True,
          return_counts=True,)

In [ ]:
n = 0
i = c.copy()


In [ ]:
n = 0

i = c.copy()
j = C[n]
l = list(zip(i,j))
dist1 = dfc[dfc.ij.isin(l)].dist.tolist()
dc = {
    'i' : i,
    'j' : j,
    'dist' : dist1}
dc


i = C[n]
j = E[n]
l = list(zip(i,j))
dist2 = [dfe[dfe.ij == ij].dist.iloc[0] for ij in l]
indiv = np.vstack([c,C[n],dist1,E[n],dist2])

rank = round(sum(dist1) + sum(dist2),2)
rank

# LATEX

In [ ]:
%%latex
$sin(x)/x$

$C^{k-1}_{n+k-1} = \frac{(n+k-1)!}{n! (k-1)!}$

In [ ]:
%%latex
$Nslot ^ {Ncapteur}$

In [ ]:
%%latex
$\sum u_{n}$ ; $\sum_{i=1}^{n} (v_{i-1}+u_{i})$ 

# Reseau

## NX

In [ ]:
plt.imshow(np.flip(A, axis=0))

In [ ]:
A = pd.read_excel('pattern.xlsx',header = None).values

A = np.flip(A.T, axis=0)
plt.imshow(A)

In [ ]:
S = list(zip(*np.where(A==1)))
C = tuple(zip(*np.where(A==2)))
PosSlots = ['S'+ str(i) for i in range(len(S))]
PosSlots = dict(zip(PosSlots,S))

PosCapt = ['C'+ str(i) for i in range(len(C))]
PosCapt = dict(zip(PosCapt,C))

PosTotal = PosSlots.copy()
PosTotal.update(PosCapt)
PosTotal

In [ ]:
comb = list(itertools.combinations(list(PosTotal.keys()),2))
dist = {}
for edge in comb:
    x1,x2 = edge
    dist[edge] = round(math.dist(PosTotal[x1],PosTotal[x2]),2)
dist

In [ ]:
G = nx.Graph()
G.add_nodes_from([(node, {'pos': v}) for (node, v) in PosTotal.items()])
# G.nodes(data=True)
for (u,v),d in dist.items():
    G.add_edge(u, v, dist=d)

fig, ax = plt.subplots(figsize = (5,7))
# nx.draw(G,  with_labels=True)
ax.imshow(A.T)
nx.draw(G, nx.get_node_attributes(G, 'pos'), with_labels=True, node_size=2000,  ax=ax)

#connectionstyle="arc3,rad=-0.3" par edge pour faire style 

## Combinatoire NEWS !!! 

In [ ]:
slot = ['S1','S2','S3']
capteurs = ['C1','C2']

In [ ]:
# combinaison reseau interne on prend pas le full connected sert a rien 
slot = list(range(Smax))
lst = [x for l in range(len(slot)) for x in itertools.combinations(ListSlots, l)]
len(lst), Smax, Smax*(Smax-1)/2
lst

In [ ]:
[l for l in range(Nslot)]

In [ ]:
n = 5
r = 2
comb = list(itertools.combinations(list(range(n)), r))
len(comb) , fact(n)/(fact(r) * fact(n-r))

In [ ]:
Nslot = 5
Ncapteur = 3
r = 2 #connexions par 2 
ListSlots = list(range(Nslot))

# connexion entre chaque slot possible = graph complete 
ListCombSlots = list(itertools.combinations(ListSlots, r))
CombMaxSlot = fact(Nslot)/(fact(r) * fact(Nslot-r))


lst = [x for l in range(Nslot) for x in itertools.combinations(ListSlots, l)]
CombMaxSlot2 = [fact(Nslot)/(fact(l) * fact(Nslot-l) ) for l in range(Nslot)]
CombMaxSlot2 = sum(CombMaxSlot2)

ListCombCapteurs = list(itertools.product(ListSlots,repeat = Ncapteur))
CombMaxCapteurs = Nslot ** Ncapteur

len(ListCombSlots) , CombMaxSlot
len(lst) ,CombMaxSlot2
len(ListCombCapteurs) , CombMaxCapteurs

# [ListCombSlots ,ListCombCapteurs]

ListCombTotal = list(itertools.product(*[ListCombSlots ,ListCombCapteurs]))
len(ListCombTotal), CombMaxSlot * CombMaxCapteurs

## save combinatoire precise des capteurs 

In [ ]:
Smax = 3
slot = list(range(Smax))
comb = list(itertools.product(slot,repeat =2))
len(comb)
a = np.array(comb)
a

In [ ]:
L= []
for i in range(Smax):
    L.append((a == i).sum(1))
counts = np.vstack(L).T  
counts

## Method on fixe les slots

In [ ]:
indiv = ['T1','T2','E1','E2']
capteurs = ['C1','C2']
lst = [indiv,capteurs]

In [ ]:

sx = list(itertools.combinations(indiv,2))
cx = list(itertools.permutations(capteurs,2))
sx
cx
comb  = list(itertools.product(*[sx,cx]))
comb

In [ ]:
pd.DataFrame(comb)

# Test nx

In [ ]:
import math
s = np.where(A==1)
import itertools 
comb = list(itertools.combinations(['S1','S2','S3'] + ['C1','C2'],2))
dist = {}
for edge in comb:
    x1,x2 = edge
    dist[edge] = round(math.dist(pos[x1],pos[x2]),2)
dist

In [ ]:
[(node, {'pos': attr}) for (node, attr) in pos.items()]

In [ ]:
G = nx.Graph()
G.add_nodes_from([(node, {'pos': attr}) for (node, attr) in pos.items()])
G.nodes(data=True)
for (u,v),d in dist.items():
    G.add_edge(u, v, dist=d)

# nx.draw(G,  with_labels=True)
nx.draw(G, nx.get_node_attributes(G, 'pos'), with_labels=True)

In [ ]:
S = tuple(zip(*np.where(A==1)))
S
C = tuple(zip(*np.where(A==2)))
S = dict(zip(['S1','S2','S3'],S))
C = dict(zip(['C1','C2'],C))
G = nx.complete_graph(S)
nx.draw(G,  with_labels=True)
G.edges
G.nodes(data=True)

In [ ]:
itertools.combinations(iterable, r)

In [ ]:
x , y = np.where(A==1)
print(np.sqrt((x1 - x0) ** 2 + (y1 - y0) ** 2))

In [ ]:
from scipy.spatial.distance import cdist
dist = cdist(s[0],s[1])

In [ ]:
from scipy.spatial.distance import euclidean
s = np.where(A==1)
s
euclidean(s[0],s[1])

In [ ]:
dict(nx.all_pairs_shortest_path(G))

In [ ]:
G = nx.Graph()
G.add_edges_from([(1,2), (1,3), (2,3), (2,4), (3,4)])
nx.draw(G,  with_labels=True)
path = nx.all_pairs_shortest_path_length(G) # This is a generator
dpath = {x[0]:x[1] for x in path} 
dpath
nx.floyd_warshall_numpy(G)

In [ ]:
distances = nx.floyd_warshall_numpy(G)
nodes = np.where(distances==2)
distances
nodes

In [ ]:
G = nx.path_graph(5)
nx.draw(G,  with_labels=True)

In [ ]:
s = np.where(A==1)
s
np.diff(s[0])

In [ ]:
np.linalg.norm(np.where(A==2))

In [ ]:
indiv = {0: {
    1: {"len": 1},
    2: {"len": 3}
}}  # single edge (0,1)
indiv
G = nx.from_dict_of_dicts(indiv)
nx.draw(G,  with_labels=True)

In [ ]:
nx.enumerate_all_cliques(G)

In [ ]:
import networkx as nx

G = nx.Graph()

# G.add_node('Hamburg', pos=(53.5672, 10.0285))
# G.add_node('Berlin', pos=(52.51704, 13.38792))
G.add_node('A', pos=(5, 1))
G.add_node('B', pos=(1, 5))
G.add_node('C', pos=(1, 3))
G.add_node('D', pos=(3, 3))
G=nx.complete_graph(['S1','S2','S3','S4'])
G = nx.complete_graph(4, nx.DiGraph())
nx.draw(G, with_labels=True)
# fig = nx.draw(G, nx.get_node_attributes(G, 'pos'), with_labels=True, node_size=0)

In [ ]:
fig

In [ ]:
pos = {city:(long, lat) for (city, (lat,long)) in nx.get_node_attributes(G, 'pos').items()}
pos
nx.draw(G, pos, with_labels=True, node_size=0)

# GA

## Import Tab

In [ ]:
edges = pd.read_excel('pattern.xlsx',sheet_name='edges')
nodes = pd.read_excel('pattern.xlsx',sheet_name='nodes')
edges.head()
nodes

In [ ]:
Cnames = ['A','B']
Enames = []

In [ ]:
c = {}
for k,v in edges.iterrows():
    if 
    

In [ ]:
Enames  = nodes[['e','mix'])].name.tolist()
Cnames = nodes[nodes.type == 'c'].name.tolist()

In [ ]:
E = nodes[nodes.Type.isin(['e','mix'])].set_index('Name')[['x','y']].to_dict(orient= 'index')
C = nodes[nodes.Type.isin(['c'])].set_index('Name')[['x','y']].to_dict(orient= 'index')
P = nodes[nodes.Type.isin(['p','mix'])].set_index('Name')[['x','y']].to_dict(orient= 'index')

In [ ]:
edges.groupby('j').i.unique()

In [ ]:

for i in edges.i.unique():
    print(i)

In [ ]:
ListNodeMix  = nodes[['e','mix'])].name.tolist()
ListCapteurs = nodes[nodes.type == 'c'].name.tolist()
Nc = len(ListCapteurs)
Ne = len(ListNodeMix)
Np = len(ListNodeMix)
c = list(range(Nc))
e = list(range(Ne))
p = list(range(Np))

C = np.random.randint(0,Ne,(10,Nc))
C

### IniPop

In [ ]:
#IniPop
C = np.random.randint(0,Ne,(10,Nc))
E = np.random.randint(0,Np,(10,Ne))
C
E

### Graph

In [ ]:
G = nx.Graph()
G.add_nodes_from(df.j.unique())
for idx , row in df.iterrows():
    u,v,d = row.values
    G.add_edge(u, v, dist=d)

In [ ]:
nx.draw(G, with_labels=True)

## INI

In [ ]:
A = pd.read_excel('pattern.xlsx',header = None).values
plt.imshow(A)

In [ ]:
from types import SimpleNamespace
dict_algo = {'pop': 10,
             'epoch': 10,
             'shortcut_pool': 0.1,
             'ChromCount': 0,
             #'Stat_T': [],
             'All_indivs': []
             }
nalgo = SimpleNamespace(**{str(k): v for k, v in dict_algo.items()})

In [ ]:
T = list(zip(*np.where(A==3)))
E = list(zip(*np.where(A==2)))
C = tuple(zip(*np.where(A==1)))
DictPos = {}
Types = ['C','E','P']
D = {}
for i in range(3):    
    Type = Types[i]
    pos = list(zip(*np.where(A==i+1)))
    names  = [Type+ str(i) for i in range(len(pos))]
    D[Type] = pos    
#     DictPos.update(dict(zip(names,pos)))
D
# DictPos

In [ ]:
Nc = len(D['C'])
Ne = len(D['E'])
Np = len(D['P'])
c = list(range(Nc))
e = list(range(Ne))
p = list(range(Np))

In [ ]:
Comb = list(itertools.product(*[c ,e]))
c_e = []
for d in Comb:
    i, j = d
    x1 = D['C'][i]
    x2 = D['E'][j]
    dist = round(math.dist(x1,x2),2)
    c_e.append([i,j,(i,j),x1,x2,dist])

Comb = list(itertools.product(*[e ,p]))
e_p = []
for d in Comb:
    i, j = d
    x1 = D['E'][i]
    x2 = D['P'][j]
    dist = round(math.dist(x1,x2),2)
    e_p.append([i,j,(i,j),x1,x2,dist])
c_e
e_p

In [ ]:
dfc = pd.DataFrame(c_e, columns = ['i','j','ij','x1','x2','dist'])
dfe = pd.DataFrame(e_p, columns = ['i','j','ij','x1','x2','dist'])
dfc
dfe

In [ ]:
C = np.random.randint(0,Ne,(10,Nc))
E = np.random.randint(0,Np,(10,Nc))
C
E

In [ ]:
n = 0

i = c.copy()
j = C[n]
l = list(zip(i,j))
dist1 = dfc[dfc.ij.isin(l)].dist.tolist()
dc = {
    'i' : i,
    'j' : j,
    'dist' : dist1}
dc


i = C[n]
j = E[n]
l = list(zip(i,j))
dist2 = [dfe[dfe.ij == ij].dist.iloc[0] for ij in l]
indiv = np.vstack([c,C[n],dist1,E[n],dist2])

rank = round(sum(dist1) + sum(dist2),2)
rank

## NX

In [ ]:
D

In [ ]:
nodes = [] 
for k,v in D.items():
    k,v
    for i in range(len(v)):
        n = (k + str(i) , {'pos' : (v[i][1], v[i][0])})
        nodes.append(n)
nodes

In [ ]:
G = nx.Graph()
G.add_nodes_from(nodes)
fig, ax = plt.subplots(figsize = (5,7))
ax.imshow(np.flip(A.T,1))
nx.draw(G, nx.get_node_attributes(G, 'pos'), with_labels=True, node_size=2000,  ax=ax)

In [ ]:
n = 0
i = C[n]
j = E[n]
l = list(zip(i,j))
i2 = ['C'+ str(i) for i in c]
j2 = ['E'+ str(i) for i in e]
l2 = list(zip(i2,j2))
dist2 = [dfe[dfe.ij == ij].dist.iloc[0] for ij in l]
for u,v in list(set(l2)):
    print(u,v)
    G.add_edge(u, v,dist = 0)

In [ ]:
fig, ax = plt.subplots(figsize = (5,7))
ax.imshow(np.flip(A.T,1))
nx.draw(G, nx.get_node_attributes(G, 'pos'), with_labels=True, node_size=1000,  ax=ax)

In [ ]:
G = nx.Graph()
G.add_nodes_from([(node, {'pos': v}) for (node, v) in PosTotal.items()])
# G.nodes(data=True)
for (u,v),d in dist.items():
    G.add_edge(u, v, dist=d)

fig, ax = plt.subplots(figsize = (5,7))
# nx.draw(G,  with_labels=True)
ax.imshow(A.T)
nx.draw(G, nx.get_node_attributes(G, 'pos'), with_labels=True, node_size=2000,  ax=ax)

## NameSpace

In [ ]:
[DictTypes['C'],DictTypes['E']]
a = DictTypes['C']
b = DictTypes['E']
Comb = list(itertools.product(*[a ,b]))
Comb
# list(itertools.combinations(Comb,2))
list(itertools.product(b,repeat=3))

In [ ]:
Nc = 3
Ne = 2
Np = 2
C = np.random.randint(0,Ne,(10,Nc))

In [ ]:
E = np.random.randint(0,Np,(10,Nc))
E

In [ ]:
C
E

In [ ]:
np.vectorize(my_dict.__getitem__)(a)

In [ ]:
i = 0
c = C[i]
e = E[i]
Rank = 


In [ ]:
pd.DataFrame(c)

In [ ]:

nmax = 5
POP = 10
pop = np.zeros((nmax, nmax), dtype= int)
np.fill_diagonal(pop, 1)
p2 = np.tile(pop,(2,1))
r = np.where(p2 == 0)
np.random.choice(r)
# np.repeat(pop,[1, 2],1)
p2
# idx = np.random.randint(0,nmax,POP)
# pop[:,idx] = 1
# pop
p2[p2 == 0]


## test

In [ ]:
c = np.random.randint(0,Ne+1,(10,Nc))

In [ ]:
c
L= []
for i in range(3):
    L.append((c == i).sum(1))
e = np.vstack(L).T  
e

In [ ]:
np.random.randint(0,Ne+1,(10,Nc))

In [ ]:
x = np.unique(c, return_counts = True)
np.array(x)

In [ ]:
dict(zip(np.array(x)))


In [ ]:
import random as rand
Nc = len()
rand.randint(0,2)

In [ ]:
nalgo

In [ ]:
SimpleNamespace()

In [ ]:
class Individual(object):
    def __init__(self, c,e,t):

#         nalgo.All_indivs.append(chrom)
        nalgo.ChromCount+=1
        #print( nalgo.ChromCount)
        self.c = chrom
        self.Rank = 0
        self.stat = {}
        self.Modif = True

    def print(self):
        print('Chrom ', self.Chrom)

    def get_rank(self):
        return self.Rank

# ANcien

## test

In [ ]:

nmax = 5
POP = 10
pop = np.zeros((nmax, nmax), dtype= int)
np.fill_diagonal(pop, 1)
p2 = np.tile(pop,(2,1))
r = np.where(p2 == 0)
np.random.choice(r)
# np.repeat(pop,[1, 2],1)
p2
# idx = np.random.randint(0,nmax,POP)
# pop[:,idx] = 1
# pop
p2[p2 == 0]


## exemple

In [ ]:
# Python3 implementation of the above approach
from random import randint

INT_MAX = 2147483647
# Number of cities in TSP
V = 5

# Names of the cities
GENES = "ABCDE"

# Starting Node Value
START = 0

# Initial population size for the algorithm
POP_SIZE = 10

# Structure of a GNOME
# defines the path traversed
# by the salesman while the fitness value
# of the path is stored in an integer


class individual:
	def __init__(self) -> None:
		self.gnome = ""
		self.fitness = 0

	def __lt__(self, other):
		return self.fitness < other.fitness

	def __gt__(self, other):
		return self.fitness > other.fitness


# Function to return a random number
# from start and end
def rand_num(start, end):
	return randint(start, end-1)


# Function to check if the character
# has already occurred in the string
def repeat(s, ch):
	for i in range(len(s)):
		if s[i] == ch:
			return True

	return False


# Function to return a mutated GNOME
# Mutated GNOME is a string
# with a random interchange
# of two genes to create variation in species
def mutatedGene(gnome):
	gnome = list(gnome)
	while True:
		r = rand_num(1, V)
		r1 = rand_num(1, V)
		if r1 != r:
			temp = gnome[r]
			gnome[r] = gnome[r1]
			gnome[r1] = temp
			break
	return ''.join(gnome)


# Function to return a valid GNOME string
# required to create the population
def create_gnome():
	gnome = "0"
	while True:
		if len(gnome) == V:
			gnome += gnome[0]
			break

		temp = rand_num(1, V)
		if not repeat(gnome, chr(temp + 48)):
			gnome += chr(temp + 48)

	return gnome


# Function to return the fitness value of a gnome.
# The fitness value is the path length
# of the path represented by the GNOME.
def cal_fitness(gnome):
	mp = [
		[0, 2, INT_MAX, 12, 5],
		[2, 0, 4, 8, INT_MAX],
		[INT_MAX, 4, 0, 3, 3],
		[12, 8, 3, 0, 10],
		[5, INT_MAX, 3, 10, 0],
	]
	f = 0
	for i in range(len(gnome) - 1):
		if mp[ord(gnome[i]) - 48][ord(gnome[i + 1]) - 48] == INT_MAX:
			return INT_MAX
		f += mp[ord(gnome[i]) - 48][ord(gnome[i + 1]) - 48]

	return f


# Function to return the updated value
# of the cooling element.
def cooldown(temp):
	return (90 * temp) / 100


# Comparator for GNOME struct.
# def lessthan(individual t1,
#			 individual t2)
# :
#	 return t1.fitness < t2.fitness


# Utility function for TSP problem.
def TSPUtil(mp):
	# Generation Number
	gen = 1
	# Number of Gene Iterations
	gen_thres = 5

	population = []
	temp = individual()

	# Populating the GNOME pool.
	for i in range(POP_SIZE):
		temp.gnome = create_gnome()
		temp.fitness = cal_fitness(temp.gnome)
		population.append(temp)

	print("\nInitial population: \nGNOME	 FITNESS VALUE\n")
	for i in range(POP_SIZE):
		print(population[i].gnome, population[i].fitness)
	print()

	found = False
	temperature = 10000

	# Iteration to perform
	# population crossing and gene mutation.
	while temperature > 1000 and gen <= gen_thres:
		population.sort()
		print("\nCurrent temp: ", temperature)
		new_population = []

		for i in range(POP_SIZE):
			p1 = population[i]

			while True:
				new_g = mutatedGene(p1.gnome)
				new_gnome = individual()
				new_gnome.gnome = new_g
				new_gnome.fitness = cal_fitness(new_gnome.gnome)

				if new_gnome.fitness <= population[i].fitness:
					new_population.append(new_gnome)
					break

				else:
					# Accepting the rejected children at
					# a possible probability above threshold.
					prob = pow(
						2.7,
						-1
						* (
							(float)(new_gnome.fitness - population[i].fitness)
							/ temperature
						),
					)
					if prob > 0.5:
						new_population.append(new_gnome)
						break

		temperature = cooldown(temperature)
		population = new_population
		print("Generation", gen)
		print("GNOME	 FITNESS VALUE")

		for i in range(POP_SIZE):
			print(population[i].gnome, population[i].fitness)
		gen += 1




mp = [
    [0, 2, INT_MAX, 12, 5],
    [2, 0, 4, 8, INT_MAX],
    [INT_MAX, 4, 0, 3, 3],
    [12, 8, 3, 0, 10],
    [5, INT_MAX, 3, 10, 0],
]
TSPUtil(mp)


# Combinatoire brut force  (a trier)

https://forums.futura-sciences.com/mathematiques-college-lycee/193661-repartir-n-objets-k-boites.html

https://fr.wikipedia.org/wiki/Combinaison_(math%C3%A9matiques)

In [ ]:

num = np.math.factorial(20)/np.math.factorial(10)
print(f"{num:,}")

In [ ]:
"qefqze : {:,.0f}".format(num)

In [ ]:
from math import factorial as f
enumerate

In [ ]:
d0 = {'Slot' : 3,'Capteur' : 4, 'elements' : 2}
d0.values()

In [ ]:
f(n)/(f(k))

A = f(n)/f(n-k)
A

In [ ]:
f(3)

In [ ]:
f(3)

In [ ]:
2**3
1**4
f(3)/f(1)

In [ ]:
slot = 2
capteur = 3

k = slot
n =capteur
n+k-1
n,k-1

a = f(n+k-1)
b = f(n) * f(k-1)
COMBINAISON = int(a/b)
print(a, b,a/b)


In [ ]:
from datetime import timedelta

def get_time_hh_mm_ss(sec):
    # create timedelta and convert it into string
    td_str = str(timedelta(seconds=sec))
    print('Time in seconds:', sec)

    # split string into individual component
    x = td_str.split(':')
    print(td_str)
    print('Time in hh:mm:ss:', x[0], 'Hours', x[1], 'Minutes', x[2], 'Seconds')

get_time_hh_mm_ss(29500000)
get_time_hh_mm_ss(num)